In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
train_instance_num, pixel_num = mnist.train.images.shape
test_instance_num, class_num = mnist.test.labels.shape

In [5]:
x = tf.placeholder('float', shape=[None, pixel_num], name='x_images')
y_answer = tf.placeholder('float', shape=[None, class_num], name='y_labels')

In [6]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) # 입력데이터를 원래 이미지의 구조로 재구성한다.
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [7]:
sess = tf.Session()

In [8]:
conv1 = tf.layers.conv2d(inputs=x_image,
                filters=32,
                kernel_size=[3,3],
                padding='same',
                activation=tf.nn.relu,
                trainable=True,
                name='conv1')

In [9]:
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
pool1

<tf.Tensor 'max_pooling2d/MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [10]:
conv2 = tf.layers.conv2d(inputs=pool1, 
                         filters=64, 
                         kernel_size=[3,3], 
                         padding='same', 
                         activation=tf.nn.relu, 
                         trainable=True, 
                         name='conv2')
conv2

<tf.Tensor 'conv2/Relu:0' shape=(?, 14, 14, 64) dtype=float32>

In [11]:
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
pool2

<tf.Tensor 'max_pooling2d_2/MaxPool:0' shape=(?, 7, 7, 64) dtype=float32>

In [12]:
pool2_flat = tf.reshape(pool2, [-1, pool2.shape[1].value*pool2.shape[2].value*pool2.shape[3].value])
pool2_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 3136) dtype=float32>

In [13]:
dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dense1

<tf.Tensor 'dense/Relu:0' shape=(?, 1024) dtype=float32>

In [14]:
dense2 = tf.layers.dense(inputs=dense1, units=10, activation=tf.nn.softmax)
dense2

<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>

In [15]:
cross_entropy = - tf.reduce_sum(y_answer*tf.log(dense2)) # 크로스 엔트로피 방식으로 오차값을 선정
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(dense2, 1), tf.argmax(y_answer, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [16]:
saver = tf.train.Saver()
saver

In [17]:
sess.run(tf.global_variables_initializer())
for i in range(1000):
    batch = mnist.train.next_batch(100) # 100 개씩 랜덤하게 픽해서 가져와 학습
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_answer:batch[1]})
        print("step %d, training accuracy : %g" % (i, train_accuracy))
        saver.save(sess, 'mnist_ckpt/total-layers-model', global_step=i, write_meta_graph=False)
    sess.run(train_step, feed_dict={x:batch[0], y_answer:batch[1]})

print("===최종학습 결과===")
print("test accuracy : %g" % sess.run(accuracy, feed_dict={x:mnist.test.images, y_answer:mnist.test.labels}))
saver.save(sess, 'mnist_ckpt/total-layers-model')

step 0, training accuracy : 0.1
step 100, training accuracy : 0.85
step 200, training accuracy : 0.88
step 300, training accuracy : 0.91
step 400, training accuracy : 0.98
step 500, training accuracy : 0.94
step 600, training accuracy : 0.96
step 700, training accuracy : 0.99
step 800, training accuracy : 0.98
step 900, training accuracy : 0.99
===최종학습 결과===
test accuracy : 0.9761


'mnist_ckpt/total-layers-model'

In [18]:
sess.close()

In [ ]:
def my_conv2d(name, shape, trainable):
    # initial = tf.truncated_normal(shape, stddev=0.1)
    conv_W = tf.get_variable(name+'_W', shape=shape, dtype=tf.float32, trainable=trainable)
    conv_b = tf.get_variable(name+'_b',)
    return tf.get_variable('conv1')